In [1]:
%matplotlib inline 

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mglearn # for visualizations

from sklearn.model_selection import train_test_split
#Cross-Validation
from sklearn.model_selection import cross_val_score, GridSearchCV

import graphviz
from sklearn.tree import export_graphviz
import pydotplus

from sklearn.datasets import load_breast_cancer

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor



## Exercise 2: Regression with random forest
### Preprocessing
A)Load the data, and consider how you want to handle missing values and categorical variables (you may choose to remove some features entirely). Carefully consider which variables are categorical. Normalize all relevant variables.

#### Theoretical explanations and decisions


In [2]:
data = pd.read_csv("flights.csv")
# We pick a treshold of missing values. In this particular case we decide that should a value have more thatn 25% missing values, it will be dropped.
data.isna().mean()>=0.25
#We make the conclusion that none of the columns are above that threshold 
data

,Unnamed: 0,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
0,1,2013,1,1,517.0,2.0,830.0,11.0,UA,N14228,1545,EWR,IAH,227.0,1400,5.0,17.0
1,2,2013,1,1,533.0,4.0,850.0,20.0,UA,N24211,1714,LGA,IAH,227.0,1416,5.0,33.0
2,3,2013,1,1,542.0,2.0,923.0,33.0,AA,N619AA,1141,JFK,MIA,160.0,1089,5.0,42.0
3,4,2013,1,1,544.0,-1.0,1004.0,-18.0,B6,N804JB,725,JFK,BQN,183.0,1576,5.0,44.0
4,5,2013,1,1,554.0,-6.0,812.0,-25.0,DL,N668DN,461,LGA,ATL,116.0,762,5.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336771,336772,2013,9,30,NaN,NaN,NaN,NaN,9E,NaN,3393,JFK,DCA,NaN,213,NaN,NaN
336772,336773,2013,9,30,NaN,NaN,NaN,NaN,9E,NaN,3525,LGA,SYR,NaN,198,NaN,NaN
336773,336774,2013,9,30,NaN,NaN,NaN,NaN,MQ,N535MQ,3461,LGA,BNA,NaN,764,NaN,NaN
336774,336775,2013,9,30,NaN,NaN,NaN,NaN,MQ,N511MQ,3572,LGA,CLE,NaN,419,NaN,NaN


### Scaling?
We may also notice that there is a large difference between the range of some variables such as dep_delay and arr_time for instance. 

However, as we will be using Decision Trees and Random Forest, which do not require feature scaling to be performed as they are not sensitive to the the variance in the data, we are skipping this part.

### In order to make the process of preprocessing data smoother we create the following helper function.
 NotHow we derive the functionality of the helper function and how we argument it could be seen in the cells bellow


In [3]:
def preprocess_inputs(df):
    df=df.copy()
    #Drop unneeded columns
    df=df.drop(['Unnamed: 0','year','tailnum','flight'],axis=1)
    #Fill remaining missing values
    remaining_na_columns=df.loc[:, df.isna().sum()>0].columns
    for column in remaining_na_columns:
        df[column] = df[column].fillna(df[column].mean())
    return df

### To help figure out which columns would be useful or not we get a list of all unique values for the dataset
### Hence we create the following dictionary

In [25]:
{column: len(data[column].unique()) for column in data.columns}

{'Unnamed: 0': 336776,
 'year': 1,
 'month': 12,
 'day': 31,
 'dep_time': 1319,
 'dep_delay': 528,
 'arr_time': 1412,
 'arr_delay': 578,
 'carrier': 16,
 'tailnum': 4044,
 'flight': 3844,
 'origin': 3,
 'dest': 105,
 'air_time': 510,
 'distance': 214,
 'hour': 26,
 'minute': 61}

We can see that there is only one unique value for year, hence we decide to drop that column

 We copuld further see that the number of unique variables for Tail Number and Flight Number are quite high. Since we are not trying to squeeze every bit of effectivness out of our data we decide to
 drop the these two categorical values in order to improve computation time when training a model on our dataset. 

In [26]:
#We also want to deal with the missing values. First we lookup all the columns where there are missing values present. We determine that all of the columns where there are missing values are
# in fact numerical
data.isna().sum()
# For simplicity purposes and due to it the fact it often results into good performence we substitue all the missing variables with the mean of the respective column


Unnamed: 0       0
year             0
month            0
day              0
dep_time      8255
dep_delay     8255
arr_time      8713
arr_delay     9430
carrier          0
tailnum       2512
flight           0
origin           0
dest             0
air_time      9430
distance         0
hour          8255
minute        8255
dtype: int64

In [4]:
flights=preprocess_inputs(data)



### After removing outliers introduce dummy variables for categorical variables

In [5]:
y = flights.loc[:,'dep_delay']
features = flights.loc[:,['air_time','arr_delay','arr_time','carrier','dep_time','dest','distance','hour','minute','month','origin']]
features = pd.get_dummies(features)
print(list(features.columns))
features[0:10]

['air_time', 'arr_delay', 'arr_time', 'dep_time', 'distance', 'hour', 'minute', 'month', 'carrier_9E', 'carrier_AA', 'carrier_AS', 'carrier_B6', 'carrier_DL', 'carrier_EV', 'carrier_F9', 'carrier_FL', 'carrier_HA', 'carrier_MQ', 'carrier_OO', 'carrier_UA', 'carrier_US', 'carrier_VX', 'carrier_WN', 'carrier_YV', 'dest_ABQ', 'dest_ACK', 'dest_ALB', 'dest_ANC', 'dest_ATL', 'dest_AUS', 'dest_AVL', 'dest_BDL', 'dest_BGR', 'dest_BHM', 'dest_BNA', 'dest_BOS', 'dest_BQN', 'dest_BTV', 'dest_BUF', 'dest_BUR', 'dest_BWI', 'dest_BZN', 'dest_CAE', 'dest_CAK', 'dest_CHO', 'dest_CHS', 'dest_CLE', 'dest_CLT', 'dest_CMH', 'dest_CRW', 'dest_CVG', 'dest_DAY', 'dest_DCA', 'dest_DEN', 'dest_DFW', 'dest_DSM', 'dest_DTW', 'dest_EGE', 'dest_EYW', 'dest_FLL', 'dest_GRR', 'dest_GSO', 'dest_GSP', 'dest_HDN', 'dest_HNL', 'dest_HOU', 'dest_IAD', 'dest_IAH', 'dest_ILM', 'dest_IND', 'dest_JAC', 'dest_JAX', 'dest_LAS', 'dest_LAX', 'dest_LEX', 'dest_LGA', 'dest_LGB', 'dest_MCI', 'dest_MCO', 'dest_MDW', 'dest_MEM', 'de

,air_time,arr_delay,arr_time,dep_time,distance,hour,minute,month,carrier_9E,carrier_AA,...,dest_STT,dest_SYR,dest_TPA,dest_TUL,dest_TVC,dest_TYS,dest_XNA,origin_EWR,origin_JFK,origin_LGA
0,227.0,11.0,830.0,517.0,1400,5.0,17.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,227.0,20.0,850.0,533.0,1416,5.0,33.0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,160.0,33.0,923.0,542.0,1089,5.0,42.0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,183.0,-18.0,1004.0,544.0,1576,5.0,44.0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,116.0,-25.0,812.0,554.0,762,5.0,54.0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
5,150.0,12.0,740.0,554.0,719,5.0,54.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
6,158.0,19.0,913.0,555.0,1065,5.0,55.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
7,53.0,-14.0,709.0,557.0,229,5.0,57.0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,140.0,-8.0,838.0,557.0,944,5.0,57.0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
9,138.0,8.0,753.0,558.0,733,5.0,58.0,1,0,1,...,0,0,0,0,0,0,0,0,0,1


Create a train test sets for subsequent training

In [6]:
X_train, X_test, y_train, y_test = train_test_split(features,y,test_size=0.20,random_state=42)

In [16]:
best_score = 0
for maximum_dept in range(13,16):
    # Set a certain number of neighbors
    dr = DecisionTreeRegressor(max_depth=maximum_dept)
    
    # Perform cross validation
    scores = cross_val_score(dr, X_train, y_train, cv=6)
    
    # Compute the mean score
    score = scores.mean()
    
    # If improvement, store score and parameter
    if score>best_score:
        best_score = score
        best_max_depth = maximum_dept

# Build a model on the combine training and valiation data
dr = DecisionTreeRegressor(max_depth=best_max_depth,random_state=0)
dr.fit(X_train, y_train)

print("Best maximum depth found: {}".format(best_max_depth))
print("Best average score: {}".format(best_score))
print("Score on training/validation set: {}".format(dr.score(X_train, y_train)))
print("Score on test set: {}".format(dr.score(X_test, y_test)))

Best maximum depth found: 13
Best average score: 0.30814312500334357
Score on training/validation set: 0.5565181617073433
Score on test set: 0.32694525618779724


In [36]:
dr = DecisionTreeRegressor(max_depth=14,random_state=0).fit(X_train, y_train)
print("Accuracy on training set: {}".format(dr.score(X_train, y_train)))
print("Accuracy on test set: {}".format(dr.score(X_test, y_test)))


Accuracy on training set: 0.9396182831654311
Accuracy on test set: 0.8989267450467417


### We visualize the decision tree

In [ ]:
plt = export_graphviz(dr, out_file=None,
                           feature_names=features.columns,
                           filled=True)

graph = graphviz.Source(dot_data)
graph

### The printed tree is rather big so we don't user pydotplus as it is hard to display anyway.

We plot the three

In [56]:
# mglearn.plots.plot_tree_partition(X_train.to_numpy(), y_train.to_numpy(),dr)

### How can we use Decision Trees for regresion analysis

The arguments regarding using Decision trees for regression vary. Decision trees as opposed to other algorithms require less effort on data preparation during pre-processing. They do not require normalization nor scaling of the data. Furthermore, missing values in the data are also not considerably crucial for building the decision tree. Decision three regressors are used to fit a sine curve, with additional noisy observations. That results in learning linear regression approximating to the sine curve. Although they are extremely good and easy to use, they tend to overfit quite easily. Therefore, proper alteration of hyperparameters is required so that the three does not learn fine details which would result in overfitting. Decision trees are especially useful when we have multiple predictors for they easily accommodate any additional predictors. They utilize residuals and compare them, therefore prediction accuracies improve because the least squared threshold sum is selected as the root of the three. That process is made for each predictor root.


In [11]:
forest = RandomForestRegressor(n_estimators=100, max_depth=50, max_features=20, random_state=0)
forest.fit(X_train, y_train)

print("Accuracy on training set: {:.3f}".format(forest.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(forest.score(X_test, y_test)))


Accuracy on training set: 0.988
Accuracy on test set: 0.920


### We use a GridSearchCV to find a good set of parameters for the regression.

In [65]:
parameter_grid = {'n_estimators':np.arange(80,100,5),'max_depth':np.arange(30,60,5),'max_features':np.arange(10,20,2), 'random_state':[0]}
forest = RandomForestRegressor()
clf = GridSearchCV(forest, parameter_grid)
clf.fit(X_train, y_train)
print(clf.best_params_)

print("Accuracy on training data = {}".format(clf.score(X_train, y_train)))
print("Accuracy on testing data = {}\n".format(clf.score(X_test, y_test)))

{'max_depth': 45, 'max_features': 18, 'n_estimators': 95, 'random_state': 0}
Accuracy on training data = 0.987409986939448
Accuracy on testing data = 0.9141701456608391



### Feature Importance 

In [11]:
rnd_rgs = RandomForestRegressor(n_estimators=95, max_depth=45, max_features=18, random_state=0)
rnd_rgs.fit(X_train, y_train)
for name, importance in zip(featuresArrivals.dtype.names, rnd_rgs.feature_importances_):
    print(name, "=", importance)

KeyboardInterrupt: 

Plot feautre importance

In [ ]:
features = featuresArrivals
importances = rnd_clf.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

### Regression analysis: Predicting arrival time delays

Training a decision tree

In [9]:
flightsArrivals=preprocess_inputs(data)

In [10]:
y = flightsArrivals.loc[:,'arr_delay']
featuresArrivals = flights.loc[:,['air_time','dep_delay','arr_time','carrier','dep_time','dest','distance','hour','minute','month','origin']]
features = pd.get_dummies(features)
print(list(features.columns))
features[0:10]

['air_time', 'arr_delay', 'arr_time', 'dep_time', 'distance', 'hour', 'minute', 'month', 'carrier_9E', 'carrier_AA', 'carrier_AS', 'carrier_B6', 'carrier_DL', 'carrier_EV', 'carrier_F9', 'carrier_FL', 'carrier_HA', 'carrier_MQ', 'carrier_OO', 'carrier_UA', 'carrier_US', 'carrier_VX', 'carrier_WN', 'carrier_YV', 'dest_ABQ', 'dest_ACK', 'dest_ALB', 'dest_ANC', 'dest_ATL', 'dest_AUS', 'dest_AVL', 'dest_BDL', 'dest_BGR', 'dest_BHM', 'dest_BNA', 'dest_BOS', 'dest_BQN', 'dest_BTV', 'dest_BUF', 'dest_BUR', 'dest_BWI', 'dest_BZN', 'dest_CAE', 'dest_CAK', 'dest_CHO', 'dest_CHS', 'dest_CLE', 'dest_CLT', 'dest_CMH', 'dest_CRW', 'dest_CVG', 'dest_DAY', 'dest_DCA', 'dest_DEN', 'dest_DFW', 'dest_DSM', 'dest_DTW', 'dest_EGE', 'dest_EYW', 'dest_FLL', 'dest_GRR', 'dest_GSO', 'dest_GSP', 'dest_HDN', 'dest_HNL', 'dest_HOU', 'dest_IAD', 'dest_IAH', 'dest_ILM', 'dest_IND', 'dest_JAC', 'dest_JAX', 'dest_LAS', 'dest_LAX', 'dest_LEX', 'dest_LGA', 'dest_LGB', 'dest_MCI', 'dest_MCO', 'dest_MDW', 'dest_MEM', 'de

,air_time,arr_delay,arr_time,dep_time,distance,hour,minute,month,carrier_9E,carrier_AA,...,dest_STT,dest_SYR,dest_TPA,dest_TUL,dest_TVC,dest_TYS,dest_XNA,origin_EWR,origin_JFK,origin_LGA
0,227.0,11.0,830.0,517.0,1400,5.0,17.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,227.0,20.0,850.0,533.0,1416,5.0,33.0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,160.0,33.0,923.0,542.0,1089,5.0,42.0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,183.0,-18.0,1004.0,544.0,1576,5.0,44.0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,116.0,-25.0,812.0,554.0,762,5.0,54.0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
5,150.0,12.0,740.0,554.0,719,5.0,54.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
6,158.0,19.0,913.0,555.0,1065,5.0,55.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
7,53.0,-14.0,709.0,557.0,229,5.0,57.0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,140.0,-8.0,838.0,557.0,944,5.0,57.0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
9,138.0,8.0,753.0,558.0,733,5.0,58.0,1,0,1,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(featuresArrivals,y,test_size=0.20,random_state=42)

In [ ]:
best_score = 0
for maximum_dept in range(13,16):
    # Set a certain number of neighbors
    dr = DecisionTreeRegressor(max_depth=maximum_dept)
    
    # Perform cross validation
    scores = cross_val_score(dr, X_train, y_train, cv=6)
    
    # Compute the mean score
    score = scores.mean()
    
    # If improvement, store score and parameter
    if score>best_score:
        best_score = score
        best_max_depth = maximum_dept

# Build a model on the combine training and valiation data
dr = DecisionTreeRegressor(max_depth=best_max_depth,random_state=0)
dr.fit(X_train, y_train)

print("Best maximum depth found: {}".format(best_max_depth))
print("Best average score: {}".format(best_score))
print("Score on training/validation set: {}".format(dr.score(X_train, y_train)))
print("Score on test set: {}".format(dr.score(X_test, y_test)))

In [ ]:
dr = DecisionTreeRegressor(max_depth=14,random_state=0).fit(X_train, y_train)
print("Accuracy on training set: {}".format(dr.score(X_train, y_train)))
print("Accuracy on test set: {}".format(dr.score(X_test, y_test)))

Training an OLS

In [ ]:
ODS = LinearRegression()
ODS.fit(X_train, y_train)

In [ ]:
print("R^2 on train data is {} and on test data is {}".format(ODS.score(X_train, y_train), 
                                                              ODS.score(X_test,y_test)))